Test By CzaOrz

In [1]:
# 导包
import json
import copy
import numpy as np

from collections import defaultdict, deque

from tree import createTree

In [2]:
"""加载数据"""
def get_data(*names):
    for name in names:
        with open('%s.json' % name, 'r', encoding='utf-8') as json_file:
            yield json.loads(json_file.read())[name]
def merge_data(*args):
    res = []
    for data in args:
        res.extend(data)
    return res
fina = merge_data(*[data for data in get_data("人事变动", "领导之窗")])

In [3]:
"""清洗数据"""

# 获取过滤目标
dupefilter = ['疑难网站', '网站异常', '驳回需求', '运维发现异常', '废弃需求', '停止维护', '网站异常']
def get_filter_spider_name(data_list, dupefilter=()):
    filter_spider = set()
    for data in data_list:
        if data['rwzt'] in dupefilter or data['spiderName'].count('-') != 2:
            filter_spider.add(data['spiderName'])
    return filter_spider
# 获取爬虫任务
def get_spider_tasks(data_list, dupefilter=()):
    spider_tasks = defaultdict(list)
    for data in data_list:
        if data['spiderName'] in dupefilter:
            continue
        spider_tasks[data['spiderName']].append(data)
    return spider_tasks
filter_spider_name = get_filter_spider_name(fina, dupefilter)
spider_tasks = get_spider_tasks(fina, filter_spider_name)

In [10]:
"""爬虫任务-开发流程模型"""
class AbnormalTask(Exception): pass
class SpiderTaskProcess:
    state = 'rwzt'
    time = 'time'
    def __init__(self, data):
        self.source = copy.deepcopy(data)
        self.data = data
        self.developer_time = 0
        self.developer_times = 0
        self.tester_time = 0
        self.tester_times = 0
        self.storager_time = 0
        self.storager_times = 0
    def statistics_dict(self):
        developer_time = self.developer_time//self.developer_times
        tester_time = self.tester_time//self.tester_times
        storager_time = self.storager_time//self.storager_times
        return dict(developer_time=developer_time,
                    developer_times=self.developer_times,
                    tester_time=tester_time,
                    tester_times=self.tester_times,
                    storager_time=storager_time,
                    storager_times=self.storager_times,
                    total_time=developer_time+tester_time+storager_time)
    def _get_state_time(self, state):
        if not isinstance(state, tuple) and not isinstance(state, list):
            state = [state]
        flag = time_use = label = None
        for index, data in enumerate(self.data):
            if data[self.state] in state:
                flag = index + 1
                time_use = data[self.time]
                try:
                    label = data[self.state]
                    if self.data[flag][self.state] in state:
                        continue
                except:
                    break
                self.data = self.data[index:]
                break
        if not flag:
            raise AbnormalTask
        return time_use, label
    def _process_task(self, start, end):
        time1, label1 = self._get_state_time(start)
        time2, label2 = self._get_state_time(end)
        return time2-time1, label2
    def check_tester_wrong(self, state1=('驳回测试修改意见', '清洗修改完成'), state2=('驳回测试修改意见')):
        try:
            return self.data[1][self.state] in state1 or \
                   self.data[2][self.state] in state2
        except:
            raise AbnormalTask
    def check_storager_wrong(self, state1=('正在入库')):
        try:
            return self.data[1][self.state] in state1
        except:
            raise AbnormalTask
    def get_developer(self, start=('开发完成','返回修改'), end=('开发完成','研发修改完成')):
        if self.check_tester_wrong():
            return self.get_tester()
        elif self.check_tester_wrong(('正在测试')):
            return self.get_tester(('正在测试'))
        elif self.check_storager_wrong():
            return self.get_storager(('返回修改'), ('入库完成'))
        else:
            time_diff, label = self._process_task(start, end)
        self.developer_time += time_diff
        self.developer_times += 1
        return label
    def get_tester(self, start=('清洗修改完成', '驳回测试修改意见'), end=('返回修改', '测试查完成')):
        time_diff, label = self._process_task(start, end)
        self.tester_time += time_diff
        self.tester_times += 1
        return label
    def get_storager(self, start=('测试查完成'), end=('返回修改', '入库完成')):
        time_diff, label = self._process_task(start, end)
        self.storager_time += time_diff
        self.storager_times += 1
        return label
    def process_label(self, label):
        if label in ['返回修改']:
            label = self.get_developer()
        elif label in ['开发完成', '研发修改完成']:
            label = self.get_tester()
        elif label in ['测试查完成']:
            label = self.get_storager()
        elif label in ['入库完成']:
            return
        self.process_label(label)
    def process(self):
        time_diff, label = self._process_task(['待处理','正在开发'], ['开发完成'])
        self.developer_time += time_diff
        self.developer_times += 1
        self.process_label(label)
        return self
# 归一化处理
minute=60
hour=minute*60
day=hour*24
two_day=day*2
five_day=day*5
def process_key_to_new_value(data, value, *keys):
    for key in keys:
        if data[key] < two_day:
            data[key] = value[0]
        elif data[key] < five_day:
            data[key] = value[1]
        else:
            data[key] = value[2]
def process_spider_task_statistics(task):
    process_key_to_new_value(task, ('1优', '2正常', '3异常'), *['developer_time','tester_time','storager_time'])
    process_key_to_new_value(task, ('good', 'normal', 'whats'), *['total_time'])
# 获取各类事件的时间
class TimerManager:
    developer_time = []
    tester_time = []
    storager_time = []
    total_time = []
    def get_merge_data(self):
        res = []
        res.extend(self.developer_time)
        res.extend(self.tester_time)
        res.extend(self.storager_time)
        return res
    def get_pointer(self):
        res = set(self.get_merge_data())
        return min(res), max(res)
    def to_normalized(self):
        min_time, max_time = self.get_pointer()
        denominator = min_time + max_time
        self.developer_time = [data/denominator for data in self.developer_time]
        self.tester_time = [data/denominator for data in self.tester_time]
        self.storager_time = [data/denominator for data in self.storager_time]
def get_each_task_time(task, labels=()): # 'developer_time', 'tester_time', 'storager_time'
    for label in labels:
        print(label, labels)
        getattr(TimerManager, label).append(task[label])

In [11]:
"""输出数据"""
labels=deque()
dataSets=[]
for spider_name, spider_task in spider_tasks.items():
    spider_task = sorted(spider_task, key=lambda task: task['time'])
    try:
        spider_task_statistics = SpiderTaskProcess(spider_task).process().statistics_dict()
    except AbnormalTask:
        continue
    get_each_task_time(spider_task_statistics, ('developer_time', 'tester_time', 'storager_time'))
    process_spider_task_statistics(spider_task_statistics)
    # process_key_to_new_value(spider_task_statistics, ('1优', '2正常', '3异常'), *['developer_time','tester_time','storager_time'])
    # process_key_to_new_value(spider_task_statistics, ('good', 'normal', 'whats'), *['total_time'])
    get_each_task_time(spider_task_statistics, ('total_time'))
    if not labels:
        for key in spider_task_statistics.keys():
            if key == 'total_time':
                labels.append(key)
            else:
                labels.appendleft(key)
    dataSet = []
    for label in labels:
        dataSet.append(spider_task_statistics.pop(label))
    dataSets.append(dataSet)
labels = list(labels)
labels.pop()
import numpy as np
np.array(dataSets).shape

developer_time ('developer_time', 'tester_time', 'storager_time')
tester_time ('developer_time', 'tester_time', 'storager_time')
storager_time ('developer_time', 'tester_time', 'storager_time')
t total_time


AttributeError: type object 'TimerManager' has no attribute 't'

In [ ]:
"""输出数据到决策树模型"""
trees = createTree(dataSets, labels)

In [ ]:
time_man = TimerManager()
time_man.to_normalized()

In [ ]:
x = range(len(time_man.developer_time))
y = time_man.developer_time
size = np.array(time_man.developer_time) * 1000
color = [random.randint(0,2) for _ in range(len(size))]
import matplotlib.pyplot as plt
plt.scatter(x, y, size, color)
plt.show()

In [ ]:
time_man.total_time